In [2]:
from elasticsearch import Elasticsearch


username = "elastic"
password = "w9nF5G7Pjt_lpVAcbqFy"

# 连接到 Elasticsearch，提供用户名和密码进行身份验证
es = Elasticsearch("http://127.0.0.1:9200", basic_auth=(username, password))
# 测试连接
if es.ping():
    print("Elasticsearch连接成功")
else:
    print("Elasticsearch连接失败")

Elasticsearch连接失败


In [4]:
import string
import random

def generate_random_string(length=10):
    characters = string.ascii_letters + string.digits
    return "".join(random.choice(characters) for _ in range(length))


index_name = generate_random_string()

def create_index(index_name, es):
    index_body = {
        "mappings": {
            "properties": {
                "title": {
                    "type": "text",
                    "analyzer": "ik_max_word",  # 使用ik分词器进行分词处理
                    "search_analyzer": "ik_smart"
                },
                "info": {
                    "type": "text",
                    "analyzer": "ik_max_word",  # 使用ik分词器进行分词处理
                    "search_analyzer": "ik_smart"
                }
            }
        }
    }
    es.indices.create(index = index_name, body = index_body)
    print(f"Index '{index_name}' created successfully.")

index_name ="book"
# 创建索引
create_index(index_name,es)


Index 'book' created successfully.


In [6]:
from db.ES.ESApi import ESApi
es1 = ESApi("elastic", "w9nF5G7Pjt_lpVAcbqFy")
# 上传文档
folder_path = "./data/books"
es1.load_es_offline_txt(index_name, folder_path)


ApiError: ApiError(500, 'null_pointer_exception', 'Cannot invoke "org.wltea.analyzer.dic.DictSegment.match(char[], int, int)" because "org.wltea.analyzer.dic.Dictionary.singleton._QuantifierDict" is null')

In [18]:
import json
# 索引名称
index_name = "book"

array = []


# 查询字符串
query = "周志明"

# 构建查询语句
body = {
    "query": {
        "multi_match": {
            "query": query,
            "fields": ["title", "info"]
        }
    },
    "size": 5
}

# 执行查询
results = es.search(index=index_name, body=body)

# 输出结果
print("Search results:")
for hit in results["hits"]["hits"]:
    print("title:", hit["_source"]["title"])
    print("info:", hit["_source"]["info"])
    array.append({"title":hit["_source"]["title"],"info":hit["_source"]["info"]})
    print(array)


Search results:
title: 智慧的疆界
info: title:智慧的疆界 rating:9.0 writer:周志明 translator:None pub_info: ISBN: series:None tags:人工智能
[{'title': '智慧的疆界', 'info': 'title:智慧的疆界 rating:9.0 writer:周志明 translator:None pub_info: ISBN: series:None tags:人工智能'}]
title: 深入理解Java虚拟机（第3版）
info: title:深入理解Java虚拟机（第3版） rating:9.5 writer:周志明 translator:None pub_info: ISBN: series:华章原创精品 tags:JVM
[{'title': '智慧的疆界', 'info': 'title:智慧的疆界 rating:9.0 writer:周志明 translator:None pub_info: ISBN: series:None tags:人工智能'}, {'title': '深入理解Java虚拟机（第3版）', 'info': 'title:深入理解Java虚拟机（第3版） rating:9.5 writer:周志明 translator:None pub_info: ISBN: series:华章原创精品 tags:JVM'}]
title: 深入理解Java虚拟机（第2版）
info: title:深入理解Java虚拟机（第2版） rating:9.0 writer:周志明 translator:None pub_info: ISBN: series:华章原创精品 tags:JVM
[{'title': '智慧的疆界', 'info': 'title:智慧的疆界 rating:9.0 writer:周志明 translator:None pub_info: ISBN: series:None tags:人工智能'}, {'title': '深入理解Java虚拟机（第3版）', 'info': 'title:深入理解Java虚拟机（第3版） rating:9.5 writer:周志明 translator:None pub_info: ISBN:

In [2]:
print("test\n\n")

test


In [4]:
from llmmodel.Prompt import Prompt

prompt = Prompt('./prompt.json').get_prompt()
print(prompt)

你是一名资深的知识库管理员，非常了解知识库的信息，但除了知识库以外的东西一无所知，你需要根据知识库提供的知识作出回答,回答问题时，你只能给出知识库的内容，不能自我发挥。
请认真学习下列信息：
    知识库内容：{0}
    请注意知识库内容中，共有五组知识，每组知识包含title和info两个字段，info字段内容是你做答的依据，你需要从五组数据从选择合适的知识进行回答，不要自我发挥，不能回答没有给出的内容。
请学习这个例子:
    回答“周志明的作品有哪些”。根据知识库 {{'title': '智慧的疆界', 'info': 'title:智慧的疆界 rating:9.0 writer:周志明 translator:None pub_info: ISBN: series:None tags:人工智能'}, {'title': '深入理解Java虚拟机（第3版）', 'info': 'title:深入理解Java虚拟机（第3版） rating:9.5 writer:周志明 translator:None pub_info: ISBN: series:华章原创精品 tags:JVM'}, {'title': '深入理解Java虚拟机（第2版）', 'info': 'title:深入理解Java虚拟机（第2版） rating:9.0 writer:周志明 translator:None pub_info: ISBN: series:华章原创精品 tags:JVM'}, {'title': '忧郁的热带', 'info': 'title:忧郁的热带 rating:9.1 writer: translator:王志明 pub_info: ISBN: series:学术前沿 tags:人类学'}, {'title': '忧郁的热带', 'info': 'title:忧郁的热带 rating:9.1 writer: translator:王志明 pub_info: ISBN: series:学术前沿 tags:人类学'}}，
    分析知识库可知，作者是周志明的书有"智慧的疆界,深入理解Java虚拟机（第3版）,深入理解Java虚拟机（第2版）"。
    那么你的回答就应该是:
    {{
             周志明的作品有智慧的疆界,深入理解Java虚拟机（

In [1]:
from db.ES.ESApi import ESApi

es = ESApi("elastic", "9YfcLUcxKiDvtzef1piK")
# 测试连接
if es.es.ping():
    print("Elasticsearch连接成功")
else:
    print("Elasticsearch连接失败")

Elasticsearch连接成功


In [2]:
import string
import random


index_name1 = es.create_index_mysql(kbname="book", info='', userid=1)

知识库创建成功！
数据库连接已关闭。
Index 'xjtdvptq17' created successfully.


In [3]:
import string
import random


index_name2 = es.create_index_mysql("law", '', 1)

知识库创建成功！
数据库连接已关闭。


Index '202mp6w1r6' created successfully.


In [4]:
# 上传文档
folder_path = "./data/books"
es.load_es_offline_txt(index_name1['kblist']['indices'], 1, folder_path)

知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据

In [5]:
# 上传文档
folder_path = "./data/law"
es.load_es_offline_csv(index_name2['kblist']['indices'], 1, folder_path)

知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据库连接已关闭。
知识创建成功！
数据